In [1]:
import urllib.request
from bs4 import BeautifulSoup

page = urllib.request.urlopen("http://www.weather.go.kr/weather/observation/currentweather.jsp").read()

soup = BeautifulSoup(page, 'html.parser')

In [2]:
import sqlite3

conn = sqlite3.connect('Weather.db')

conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [3]:
# <input type="text" id="observation_text" class="time" name="tm" value="2020. 09. 28. 09:00">
current_time = soup.find('input', {'class':'time'})

dt = current_time['value'].split('.')
year = dt[0]
month = dt[1]
date = dt[2]
time = dt[3]

date_time = year+"-"+month+"-"+date+" "+time 
print(date_time)

2020-09-28 11:00


In [4]:
table = soup.find('table', {'class': 'table_develop3'})

data = []

for tr in table.find_all('tr'):
    tds = list(tr.find_all('td'))
    for td in tds:
        if td.find('a'):
            city = td.find('a').text
            now = tds[1].text
            temperature = tds[5].text
            humidity = tds[9].text
            atm = tds[12].text
            data.append([city, now, temperature, humidity, atm])
           
            cursor.execute('''
                insert into CityWeather (city, now, temperature, humidity, atm, dtime) values(?, ?, ?, ?, ?, ?)''',
                           (city, now, temperature, humidity, atm, date_time))

conn.commit()

In [5]:
## 표준기압(atm)=760mmHg=1,013.25hPa
cursor.execute('select * from CityWeather where now == "맑음" and temperature > 18 and atm > 1015')
rows = cursor.fetchall()

for row in rows:
    print("도시: %s, 현재일기: %s 온도: %s, 습도: %s, 기압: %s, 시간: %s" % 
              (row['city'], row['now'], row['temperature'], row['humidity'], row['atm'], row['dtime']))

conn.close()